In [ ]:
import pandas as pd
df = pd.read_excel("your_file.xlsx")
df.head()

In [ ]:
# Add helper columns
df['Has_ICD'] = df["ICD or Mapping Available (Yes/No)"].str.strip().str.lower() == "yes"
df['In_Prod'] = df["Matching with Prod"].str.lower().str.contains("prod", na=False)
df['Is_P1'] = df["Priority"].str.strip().str.upper() == "P1"
df['Remarks'] = df['Remarks'].fillna("")

In [ ]:
# Step 3: ICD by Platform
icd_by_platform = df.groupby(["Platform", "Has_ICD"]).size().unstack(fill_value=0)
for col in [True, False]:
    if col not in icd_by_platform.columns:
        icd_by_platform[col] = 0
icd_by_platform["Total"] = icd_by_platform[True] + icd_by_platform[False]
icd_by_platform["% ICD Available"] = (icd_by_platform[True] / icd_by_platform["Total"].replace(0, 1) * 100).round(2)
icd_by_platform

In [ ]:
# Step 4: P1 services not in Prod or missing ICD
p1_risk = df[df["Is_P1"] & (~df["Has_ICD"] | ~df["In_Prod"])]
p1_risk

In [ ]:
# Step 5: Deployment summary
deployment_summary = df["Matching with Prod"].value_counts(dropna=False).rename_axis("Deployment Status").reset_index(name="Count")
deployment_summary

In [ ]:
# Step 6: File formats
file_format_counts = df["FileFormat"].value_counts(dropna=False).reset_index()
file_format_counts.columns = ["File Format", "Count"]
file_format_counts

In [ ]:
# Step 7: Partners sent
partner_counts = df["Partners Sent"].value_counts(dropna=False).reset_index()
partner_counts.columns = ["Partner", "Count"]
partner_counts

In [ ]:
# Step 8: Remarks keyword detection
keywords = ["descoped", "mismatch", "handoff", "encryption", "feed", "issue", "delay", "error"]
df["Keyword_Flag"] = df["Remarks"].str.contains("|".join(keywords), case=False)
remarks_issues = df[df["Keyword_Flag"]]
remarks_issues[["Platform", "CapabilityName", "Remarks"]]

In [ ]:
# Step 9: Remarks by platform
remarks_by_platform = remarks_issues.groupby("Platform").size().reset_index(name="Flagged Remarks")
remarks_by_platform

In [ ]:
# Step 10: Capability vs Country
capability_country = df.pivot_table(index="CapabilityName", columns="Country Code", values="Service_ID", aggfunc="count", fill_value=0)
capability_country

In [ ]:
# Step 11: Risk Levels
df['Risk_Level'] = df.apply(lambda row: (
    "🔥 Critical" if row["Is_P1"] and (not row["Has_ICD"] or not row["In_Prod"]) else
    "⚠ Warning" if not row["Has_ICD"] or not row["In_Prod"] else
    "✅ Healthy"
), axis=1)
remediation_plan = df[["Service_ID", "Platform", "CapabilityName", "Country Code", "Priority", "Risk_Level", "Remarks"]].sort_values(by="Risk_Level")
remediation_plan

In [ ]:
# Step 12: Platform efficiency score
platform_scores = df.groupby("Platform").agg(
    Total_Services=("Service_ID", "count"),
    Completed_ICDs=("Has_ICD", "sum"),
    Prod_Services=("In_Prod", "sum"),
    P1_Count=("Is_P1", "sum"),
)
platform_scores["Efficiency_Score"] = (
    (platform_scores["Completed_ICDs"] / platform_scores["Total_Services"]) * 0.4 +
    (platform_scores["Prod_Services"] / platform_scores["Total_Services"]) * 0.4 +
    ((platform_scores["Total_Services"] - platform_scores["P1_Count"]) / platform_scores["Total_Services"]) * 0.2
).round(2)
platform_scores.sort_values(by="Efficiency_Score", ascending=False)

In [ ]:
# Step 13: UAT only services
uat_only = df[df["Matching with Prod"].str.lower().str.contains("uat", na=False) & ~df["In_Prod"]]
uat_only[["Service_ID", "Platform", "CapabilityName", "Country Code", "Remarks"]]

In [ ]:
# Step 14: Top risk words in remarks
from collections import Counter
import re
all_remarks = " ".join(df["Remarks"].dropna()).lower()
words = re.findall(r'\b[a-z]{4,}\b', all_remarks)
common_words = Counter(words).most_common(10)
common_words

In [ ]:
# Step 15: Charts
import matplotlib.pyplot as plt
df["Risk_Level"].value_counts().plot(kind="pie", autopct="%1.1f%%", title="Risk Distribution", ylabel="")
plt.show()
icd_by_platform[[True, False]].plot(kind="bar", stacked=True, title="ICD Availability by Platform")
plt.ylabel("Service Count")
plt.show()
remarks_by_platform.set_index("Platform").plot(kind="bar", title="Remarks-Flagged Issues by Platform")
plt.ylabel("Issue Count")
plt.show()